## Video Game Character Project
#### Part I:
Apply what you have learned in the previous lectures to insert and find data related to your own video game characters within your MongoDB database.

#### Import Dependencies

In [3]:
import copy
import numpy as np

from pymongo import MongoClient

#### Player and Items Python Classes 

In [12]:
class Player:
    def __init__(self, name, max_health, max_energy, items=[]):
        self.name = name
        self.health = max_health 
        self.max_health = max_health
        self.energy = max_energy
        self.max_energy = max_energy
        self.items = copy.deepcopy(items)
        
        
    def attack(self, player):
        energy_cost = 5
        
        if self.energy >= energy_cost:
            attack_strength = np.random.randint(1, 6)
            player.health -= attack_strength
            self.energy -= energy_cost
            print("{} attacked {} for {} damage".format(self.name, player.name, attack_strength))
        else:
            print("{} does not have enough energy to attack {}".format(self.name, player.name))
        
        
    def heal(self, amount):
        self.health += amount
        
        if self.health > self.max_health:
            self.health = self.max_health
         
        
    def stats(self):
        return vars(self)
        
        
    def use_item(self, item_name):
        try: 
            item = next(item for item in self.items if item.name == item_name)
            item.quantity -= 1

            for effect in item.effects:

                for method, value in effect.items():
                    class_method = getattr(self, method)
                    class_method(value)

            if item.quantity == 0:
                self.items.remove(item)
                
        except:
            print("{} does not have any {}s".format(self.name, item_name))

In [13]:
class Item:
    def __init__(self, name, quantity, effects=[]):
        self.name = name
        self.quantity = quantity 
        self.effects = effects
        
        
    def __repr__(self):
        return "Item(name={}, quantity={}, effects={})".format(self.name, self.quantity, self.effects)

#### Playground Area
Feel free to play around with the Player and Item classes defined above to get a feel for how they work.

In [14]:
# Example usage of Item:
potion = Item("health_potion", 2, [{"heal": 10}])
player_obj = Player("kevin", 12, 200, [potion])

In [15]:
player_obj.stats()

{'name': 'kevin',
 'health': 12,
 'max_health': 12,
 'energy': 200,
 'max_energy': 200,
 'items': [Item(name=health_potion, quantity=2, effects=[{'heal': 10}])]}

#### Project Instructions

In [16]:
# TODO:
# 1) Initalize a MongoDB Client object to connect to your database with
client = MongoClient("mongodb://localhost:27017")
db = client["video_game"]
player_col = db["player"]

In [28]:
# TODO:
# 2) Create a function that takes in a Player object and inserts it into the database, 
#    Extra Challenge: check for duplicate player entries, if so, do not insert again
def insert_player(player_obj, check_for_duplicate=True):
    if check_for_duplicate:
        duplicate_entry = player_col.find_one({"name": player_obj.name})
        if duplicate_entry != None:
            return duplicate_entry["_id"]
    player_dict = copy.deepcopy(vars(player_obj))
    player_items_dict = []
    for item in player_dict["items"]:
        items_dict = vars(item)
        player_items_dict.append(items_dict)
    player_dict["items"] = player_items_dict
    return player_col.insert_one(player_dict).inserted_id

# 3) Create a function that is able to find a Player in the databse by searching for their name
def find_player_by_name(player_name):
    return db.player.find_one({"name": player_name})

# 4) Create a function that loads the data from the above function and returns a Player object configured with that data
def convert_player_object(player_obj):
    p = player_obj
    items = []
    print(p)
    for item in p["items"]:
        items.append(Item(item["name"], item["quantity"], item["effects"]))
    player_obj = Player(p["name"], p["max_health"], p["max_energy"], items)
    player_obj.health = p["health"]
    player_obj.energy = p["energy"]
    return player_obj

def find_player_obj_by_name(name):
    player = find_player_by_name(name)
    return convert_player_object(player)


In [ ]:
# TODO:
# 5) Create at least 2 players, optionally give them items
player1_potion = [Item("curse", quantity=2, effects=[{"damage": 10}])]
player2_potion = [Item("dark magic", quantity=1, effects=[{"damage": 20}])]
player3_potion = [Item("Potion", quantity=1, effects=[{"heal": 20}])]
player1 = Player("Maximus", 12, 200, player1_potion)
player2 = Player("sofe", 12, 200, player2_potion)

# 6) Insert Players into MongoDB
insert_id_1 = insert_player(player1)
insert_id_2 = insert_player(player2)
print(insert_id_1)
print(insert_id_2)
# 7) Load the player data from MongoDB into new player variables


67dfe03592fd3ce3514bc685
67dfe03692fd3ce3514bc686


TypeError: 'NoneType' object is not subscriptable

In [32]:
player2_load = find_player_obj_by_name("sofe")

{'_id': ObjectId('67dfe03692fd3ce3514bc686'), 'name': 'sofe', 'health': 12, 'max_health': 12, 'energy': 200, 'max_energy': 200, 'items': [{'name': 'dark magic', 'quantity': 1, 'effects': [{'damage': 20}]}]}


In [34]:
player2_load.items

[Item(name=dark magic, quantity=1, effects=[{'damage': 20}])]

In [35]:
player_col.delete_many({})

DeleteResult({'n': 2, 'ok': 1.0}, acknowledged=True)